In [1]:
!pip install optuna -q

In [44]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score

import optuna

In [45]:
X_test_mat100 = pd.read_csv('../data/Xte_mat100.csv',sep=' ',header=None).values
X_train_mat100 = pd.read_csv('../data/Xtr_mat100.csv',sep=' ',header=None).values


# X_test = pd.read_csv('../data/Xte.csv',sep=',',index_col=0).values
# X_train = pd.read_csv('../data/Xtr.csv',sep=',',index_col=0).values

y = pd.read_csv('../data/Ytr.csv',sep=',',index_col=0).values

In [46]:
print('x_train: {} y_train {}'.format(X_train_mat100.shape,y.shape))
print('x_train: {} y_train {}'.format(X_train.shape,y.shape))
print('x_test: {}'.format(X_test.shape))

x_train: (2000, 100) y_train (2000, 1)
x_train: (1340, 100) y_train (2000, 1)


NameError: name 'X_test' is not defined

In [47]:
X_train_mat100 = scale(X_train_mat100)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_mat100, y, test_size=0.33, random_state=42)

print(X_train.shape,X_val.shape,y_train.shape, y_val.shape)

(1340, 100) (660, 100) (1340, 1) (660, 1)


### Model Test

In [48]:
class logisticregression():
    def __init__(self,train_data,train_labels,lamda=0.2,lr=0.01,batch_size=None,epoch=10,print_every = 10):
        dummy_once = np.ones((len(train_data),1))
        self.train_data = np.hstack((dummy_once,train_data))
        self.train_labels = train_labels
        
        self.params = np.zeros((len(self.train_data[0]),1))
        
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.print_every = print_every
        self._lambda = lamda
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def cost(self,y,y_pred):
        return -np.mean(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    def gradient(self,y,y_pred,x):
        return np.dot(x.T,(y_pred-y))#+(2*self._lambda*self.params)
    
    def train(self):
        for i in range(self.epoch):
            y_pred = self.sigmoid(np.dot(self.train_data,self.params))
            loss = self.cost(self.train_labels,y_pred)
            
            gra = self.gradient(self.train_labels,y_pred,self.train_data)
            
            self.params -= self.lr*gra
            
            if self.print_every:
                if i%self.print_every == 0 or i == self.epoch-1:
                    print('Epoch : {}  Loss: {}'.format(i,loss))
    def predict(self,test_data):
        result = self.sigmoid(np.dot(test_data,self.params[1:])+self.params[0])
        result[result > 0.5 ] = 1
        result[result <= 0.5 ] = 0
        return result
    
    def evaluate(self,test_data,labels):
        accuracy = accuracy_score(self.predict(test_data),labels)
        return accuracy

In [49]:
def cross_validate(x_data,y_data,lr,k=5):
    if len(x_data)%k != 0:
        print('cant vsplit',len(x_data),' by ',k)
        return
    
    x_data_splitted = np.vsplit(x_data,k)
    y_data_splitted = np.vsplit(y_data,k)
    
    aggrigate_result = []
    for i in range(len(x_data_splitted)):
        train = []
        test = []
        items = [j for j in range(len(x_data_splitted)) if j !=i ]
        x_test = x_data_splitted[i]
        y_test = y_data_splitted[i]
        for item in items:
            if len(train) == 0:
                x_train = x_data_splitted[item]
                y_train = y_data_splitted[item]
            else:
                x_train = np.concatenate((x_train,x_data_splitted[item]), axis=0)
                y_train = np.concatenate((y_train,y_data_splitted[item]), axis=0)
            
        logistic = logisticregression(x_train,y_train,lr=lr,print_every=None)
        logistic.train()
        
        result = logistic.evaluate(x_test,y_test)
        aggrigate_result.append(result)
        
    return sum(aggrigate_result)/len(aggrigate_result)

In [50]:
def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    k =  trial.suggest_int('k', 2, 10,2)
#     epoch =  trial.suggest_int('epoch', 3, 10)
    return cross_validate(X_train_mat100, y,lr,k)

In [51]:
# cross_validate(X_train_mat100, y,0.001,10)

import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective, n_trials=20,show_progress_bar=True)

/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



[I 2020-05-23 23:45:42,638] Finished trial#0 with value: 0.5795 with parameters: {'lr': 2.3726062080849826e-05, 'k': 4}. Best is trial#0 with value: 0.5795.
[I 2020-05-23 23:45:42,772] Finished trial#1 with value: 0.5725 with parameters: {'lr': 1.5427931956440158e-05, 'k': 8}. Best is trial#0 with value: 0.5795.
[I 2020-05-23 23:45:42,892] Finished trial#2 with value: 0.579 with parameters: {'lr': 6.26612582178828e-05, 'k': 4}. Best is trial#0 with value: 0.5795.
[I 2020-05-23 23:45:43,023] Finished trial#3 with value: 0.5605 with parameters: {'lr': 0.0025264875230715583, 'k': 10}. Best is trial#0 with value: 0.5795.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in multiply



[I 2020-05-23 23:45:43,139] Finished trial#4 with value: 0.5730000000000001 with parameters: {'lr': 4.738224281790927e-05, 'k': 8}. Best is trial#0 with value: 0.5795.
[I 2020-05-23 23:45:43,251] Finished trial#5 with value: 0.5335000000000001 with parameters: {'lr': 0.01972542720998966, 'k': 8}. Best is trial#0 with value: 0.5795.
[I 2020-05-23 23:45:43,370] Finished trial#6 with value: 0.5615 with parameters: {'lr': 0.0005650868206767763, 'k': 10}. Best is trial#0 with value: 0.5795.
[I 2020-05-23 23:45:43,487] Finished trial#7 with value: 0.5605 with parameters: {'lr': 0.0038021188498867575, 'k': 10}. Best is trial#0 with value: 0.5795.
[I 2020-05-23 23:45:43,606] Finished trial#8 with value: 0.541 with parameters: {'lr': 0.004974158036256035, 'k': 8}. Best is trial#0 with value: 0.5795.
[I 2020-05-23 23:45:43,715] Finished trial#9 with value: 0.597 with parameters: {'lr': 0.0006958943246111198, 'k': 2}. Best is trial#9 with value: 0.597.
[I 2020-05-23 23:45:43,851] Finished trial#1

In [52]:
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
# df = df.dropna()
df.sort_values(by=['value'])

,number,value,duration,params_k,params_lr,system_attrs_fail_reason
5,5,0.5335,00:00:00.106649,8,0.019725,NaN
8,8,0.5410,00:00:00.113214,8,0.004974,NaN
15,15,0.5490,00:00:00.107787,4,0.042739,NaN
3,3,0.5605,00:00:00.124802,10,0.002526,NaN
7,7,0.5605,00:00:00.112463,10,0.003802,NaN
6,6,0.5615,00:00:00.112949,10,0.000565,NaN
1,1,0.5725,00:00:00.128792,8,0.000015,NaN
4,4,0.5730,00:00:00.110628,8,0.000047,NaN
13,13,0.5780,00:00:00.105824,4,0.000123,NaN
2,2,0.5790,00:00:00.115134,4,0.000063,NaN


In [80]:
cross_validate(X_train_mat100, y,0.000307,k=4)

0.582

In [81]:
# X_train, X_test, y_train, y_test
logistic = logisticregression(X_train,y_train,epoch=10,print_every=1,lr=0.000307)
logistic.train()
        
print(logistic.evaluate(X_train,y_train))
print(logistic.evaluate(X_val,y_val))

Epoch : 0  Loss: 0.6931471805599453
Epoch : 1  Loss: 0.6710256163735592
Epoch : 2  Loss: 0.6569867825131193
Epoch : 3  Loss: 0.6471536929672367
Epoch : 4  Loss: 0.6399731551874114
Epoch : 5  Loss: 0.6345373589277915
Epoch : 6  Loss: 0.6302931056826069
Epoch : 7  Loss: 0.6268916086445367
Epoch : 8  Loss: 0.62410535055381
Epoch : 9  Loss: 0.6217810466087367
0.6477611940298508
0.6015151515151516


In [82]:
X_test_mat100 = pd.read_csv('../data/Xte_mat100.csv',sep=' ',header=None).values

X_test_mat100[0,:]

X_test_mat100 = scale(X_test_mat100)

In [83]:

sumbission = []
for i in range(len(X_test_mat100)):
    result = logistic.predict(X_test_mat100[i])
    sumbission.append([i,int(result)])
    result

In [84]:
# sumbission
df = pd.DataFrame(sumbission)
df.columns = ['Id','Bound']
df.to_csv('test.csv',index=False)

In [85]:
df.head(50)

,Id,Bound
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0
5,5,1
6,6,0
7,7,1
8,8,1
9,9,1
